In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time

def scrape_premier_league_season(url, season):
    """
    Extrae la tabla de clasificación de una temporada de Premier League desde Wikipedia
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Estrategia 1: Buscar tabla después del encabezado "Clasificación"
        clasificacion_table = None
        
        # Buscar todos los encabezados h2, h3 que contengan "Clasificación" o "Tabla"
        for heading in soup.find_all(['h2', 'h3', 'span'], class_='mw-headline'):
            heading_text = heading.get_text().strip()
            if 'Clasificación' in heading_text or 'Tabla' in heading_text:
                # Buscar la siguiente tabla después de este encabezado
                parent = heading.find_parent(['h2', 'h3'])
                if parent:
                    next_table = parent.find_next('table', {'class': 'wikitable'})
                    if next_table:
                        clasificacion_table = next_table
                        break
        
        # Estrategia 2: Si no se encuentra, buscar tabla que tenga columnas típicas de clasificación
        if not clasificacion_table:
            tables = soup.find_all('table', {'class': 'wikitable'})
            for table in tables:
                header_row = table.find('tr')
                if header_row:
                    headers_text = header_row.get_text()
                    # Verificar si tiene columnas típicas de clasificación
                    if ('Pts' in headers_text or 'Puntos' in headers_text) and \
                       ('PJ' in headers_text or 'Partidos' in headers_text) and \
                       ('GF' in headers_text or 'Goles' in headers_text):
                        rows = table.find_all('tr')
                        if len(rows) >= 20:  # Debe tener al menos 20 equipos
                            clasificacion_table = table
                            break
        
        if not clasificacion_table:
            return None
        
        # Extraer los datos
        rows = clasificacion_table.find_all('tr')
        
        # Extraer encabezados
        header_row = rows[0]
        headers = []
        for th in header_row.find_all(['th', 'td']):
            text = th.get_text().strip()
            text = re.sub(r'\s+', ' ', text)
            text = re.sub(r'\[.*?\]', '', text)
            headers.append(text)
        
        # Verificar que sean los encabezados correctos
        headers_str = ' '.join(headers)
        if not ('Pts' in headers_str or 'Puntos' in headers_str):
            return None
        
        print(f"✓ Encabezados: {headers[:11]}")
        
        # Extraer datos
        data = []
        for row in rows[1:]:
            cols = row.find_all(['td', 'th'])
            if len(cols) >= 10:  # Debe tener al menos las columnas básicas
                row_data = []
                for i, col in enumerate(cols):
                    # Para la columna del equipo (normalmente posición 1)
                    if i == 1:
                        link = col.find('a')
                        if link and link.get('title'):
                            text = link.get('title')
                        else:
                            text = col.get_text().strip()
                    else:
                        text = col.get_text().strip()
                    
                    # Limpiar
                    text = re.sub(r'\s+', ' ', text)
                    text = re.sub(r'\[.*?\]', '', text)
                    text = text.replace('↑', '').replace('↓', '').strip()
                    text = text.replace('(C)', '').replace('(D)', '').strip()
                    
                    row_data.append(text)
                
                if row_data and len(row_data) >= 10:
                    data.append(row_data)
        
        if not data or len(data) < 15:  # Debe haber al menos 15 equipos
            return None
        
        # Crear DataFrame
        df = pd.DataFrame(data)
        
        # Asignar nombres de columnas estándar
        standard_headers = ['Pos', 'Equipo', 'Pts', 'PJ', 'G', 'E', 'P', 'GF', 'GC', 'Dif']
        df.columns = standard_headers[:len(df.columns)] if len(df.columns) <= 10 else headers[:len(df.columns)]
        
        # Limpiar la columna de posición
        if 'Pos' in df.columns:
            df['Pos'] = df['Pos'].str.extract(r'(\d+)')[0]
        
        # Agregar columna de temporada
        df['Temporada'] = season
        
        return df
        
    except Exception as e:
        print(f"  Error: {str(e)}")
        return None


def scrape_all_seasons(start_year=1992, end_year=2025):
    """
    Extrae todas las temporadas de Premier League
    """
    all_data = []
    failed_seasons = []
    
    print("="*70)
    print("EXTRACCIÓN DE DATOS DE PREMIER LEAGUE - WIKIPEDIA")
    print("="*70)
    
    for year in range(start_year, end_year):
        next_year = year + 1
        season = f"{year}-{str(next_year)[-2:]}"
        url = f"https://es.wikipedia.org/wiki/Premier_League_{season}"
        
        print(f"\n[{year-start_year+1}/{end_year-start_year}] Temporada {season}...", end=" ")
        
        df = scrape_premier_league_season(url, season)
        
        if df is not None and len(df) >= 15:
            all_data.append(df)
            print(f"✓ {len(df)} equipos extraídos")
        else:
            print(f"✗ FALLÓ")
            failed_seasons.append(season)
        
        # Pausa para no sobrecargar Wikipedia
        time.sleep(0.5)
    
    print("\n" + "="*70)
    print("RESUMEN")
    print("="*70)
    print(f"✓ Temporadas exitosas: {len(all_data)}")
    print(f"✗ Temporadas fallidas: {len(failed_seasons)}")
    
    if failed_seasons:
        print(f"\nTemporadas que fallaron: {', '.join(failed_seasons)}")
        print("\nEstas temporadas pueden requerir extracción manual o tienen")
        print("un formato diferente en Wikipedia.")
    
    # Combinar todos los datos
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        
        # Guardar archivo combinado
        combined_df.to_csv('premier_league_completo.csv', index=False, encoding='utf-8-sig')
        print(f"\n✓ Archivo guardado: 'premier_league_completo.csv'")
        print(f"  Total de registros: {len(combined_df)}")
        print(f"  Temporadas incluidas: {combined_df['Temporada'].nunique()}")
        
        # Guardar también por temporada individual
        for season_df in all_data:
            season = season_df['Temporada'].iloc[0]
            filename = f'premier_league_{season}.csv'
            season_df.to_csv(filename, index=False, encoding='utf-8-sig')
        
        print(f"\n✓ También se guardaron {len(all_data)} archivos individuales")
        
        # Mostrar estadísticas
        print("\n" + "="*70)
        print("ESTADÍSTICAS")
        print("="*70)
        print("\nEquipos por temporada:")
        teams_by_season = combined_df.groupby('Temporada').size()
        for season, count in teams_by_season.items():
            print(f"  {season}: {count} equipos")
        
        return combined_df, failed_seasons
    else:
        print("\n✗ No se pudieron extraer datos")
        return None, failed_seasons


def extract_failed_seasons_manually(failed_seasons):
    """
    Intenta extraer las temporadas fallidas con estrategias alternativas
    """
    print("\n" + "="*70)
    print("REINTENTANDO TEMPORADAS FALLIDAS")
    print("="*70)
    
    recovered_data = []
    
    for season in failed_seasons:
        year = int(season.split('-')[0])
        next_year = year + 1
        
        print(f"\nTemporada {season}:")
        
        # Intentar URL en inglés
        url_en = f"https://en.wikipedia.org/wiki/{year}–{str(next_year)[-2:]}_Premier_League"
        print(f"  Intentando versión inglesa...")
        
        headers = {'User-Agent': 'Mozilla/5.0'}
        try:
            response = requests.get(url_en, headers=headers)
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Buscar tabla de clasificación en inglés
            tables = soup.find_all('table', {'class': 'wikitable'})
            for table in tables:
                header_text = table.get_text()
                if 'Pts' in header_text and 'GF' in header_text:
                    rows = table.find_all('tr')
                    if len(rows) >= 20:
                        print(f"  ✓ Encontrada en Wikipedia inglesa")
                        df = scrape_premier_league_season(url_en, season)
                        if df is not None:
                            recovered_data.append(df)
                            break
        except:
            print(f"  ✗ No disponible en inglés")
    
    return recovered_data


# EJECUCIÓN PRINCIPAL
if __name__ == "__main__":
    # Extraer todas las temporadas
    df_complete, failed = scrape_all_seasons(1992, 2025)
    
    # Intentar recuperar las temporadas fallidas
    if failed:
        print("\n¿Deseas intentar extraer las temporadas fallidas de Wikipedia en inglés?")
        print("Esto puede ayudar a completar los datos faltantes.")
        recovered = extract_failed_seasons_manually(failed)
        
        if recovered and df_complete is not None:
            df_complete = pd.concat([df_complete] + recovered, ignore_index=True)
            df_complete.to_csv('premier_league_completo.csv', index=False, encoding='utf-8-sig')
            print(f"\n✓ Se recuperaron {len(recovered)} temporadas adicionales")
    
    print("\n" + "="*70)
    print("✅ PROCESO COMPLETADO")
    print("="*70)

EXTRACCIÓN DE DATOS DE PREMIER LEAGUE - WIKIPEDIA

✓ Encabezados: ['Pos.', 'Equipo', 'Pts.', 'PJ', 'G', 'E', 'P', 'GF', 'GC', 'Dif.', 'Clasificación 1993-94']
✓ 22 equipos extraídos

✓ Encabezados: ['Pos.', 'Equipo', 'Pts.', 'PJ', 'G', 'E', 'P', 'GF', 'GC', 'Dif.', 'Clasificación 1994-95']
✓ 22 equipos extraídos

✓ Encabezados: ['Pos.', 'Equipo', 'Pts.', 'PJ', 'G', 'E', 'P', 'GF', 'GC', 'Dif.', 'Clasificación 1995-96']
✓ 22 equipos extraídos

✓ Encabezados: ['Pos.', 'Equipo', 'Pts.', 'PJ', 'G', 'E', 'P', 'GF', 'GC', 'Dif.', 'Clasificación 1996-97']
✓ 20 equipos extraídos

✓ Encabezados: ['Pos.', 'Equipo', 'Pts.', 'PJ', 'G', 'E', 'P', 'GF', 'GC', 'Dif.', 'Clasificación 1997-98']
✓ 20 equipos extraídos

✓ Encabezados: ['Pos.', 'Equipo', 'Pts.', 'PJ', 'G', 'E', 'P', 'GF', 'GC', 'Dif.', 'Clasificación 1998-99']
✓ 20 equipos extraídos

✓ Encabezados: ['Pos.', 'Equipo', 'Pts.', 'PJ', 'G', 'E', 'P', 'GF', 'GC', 'Dif.', 'Clasificación 1999-00']
✓ 20 equipos extraídos

✗ FALLÓTemporada 1999-00.